In [4]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [17]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920x1080")
driver = webdriver.Chrome(chrome_options=chrome_options,
                          executable_path='chromedriver.exe')

fname = 'liontravel_routes.txt'
with open(fname, encoding='utf8') as f:
    content = f.readlines()

urls = list(set([x.strip() for x in content if not x.startswith('#')]))

columns=['團型名稱', '團號', '價錢', '出國日期', '抵台日期',
         '剩餘空位', '坐位總數', '出國航空', '出國時間', '出國機場',
         '抵達時間', '抵達機場', '回國航空', '回國時間', '回國機場',
         '抵台時間', '抵台機場', '網址']
df = pd.DataFrame(data=None, columns=columns)

In [18]:
def df_filter():
    def tt(t):
        if t.startswith('團號:'):
            return t[3:]
    def t2(t):
        if t.startswith('"'):
            t = t[1:]
        if t.endswith('"'):
            t = t[:-1]
        return t
    df['團號'] = df['團號'].apply(tt)
    df['團型名稱'] = df['團型名稱'].apply(t2)

def save_to_local():
    df.to_csv('lion_travel.csv', encoding='utf-8-sig',index=False)

current_index = 0
trys = 0
while True:
    if trys == 10:
        driver.close()
        driver.quit()
        save_to_local()
        break
    for index, url in enumerate(urls):
        if index < current_index:
            continue
        print(f"{index+1}/{len(urls)}")
        driver.get(url)
        time.sleep(6)
        elem = driver.find_element_by_class_name("tourTopInfo")
        source_code = elem.get_attribute('innerHTML')
        
        if '查詢中...' in source_code:
            current_index = index
            trys +=1
            break

        soup = BeautifulSoup(source_code, 'html.parser')

        try:
            # 團型名稱
            JSON = re.compile('ProductName: ".*?"', re.DOTALL)
            trip_name = JSON.search(source_code).group(0).split()[1]
        except IndexError:
            current_index = index
            trys +=1
            break

        # 團號
        trip_number = soup.find_all('th', {'class':'tptitle'})[0].span.text

        # 價錢
        price = soup.find('span', {'class':'price'}).text

        tpinfos = soup.find_all('tr', {'class':'tpinfo'})

        # 出發日期
        start_date = tpinfos[0].find_all('td')[1].text

        # 抵達日期
        back_date = tpinfos[1].find_all('td')[1].text

        # 團位數量
        empty_seats = tpinfos[2].find_all('span')[0].text
        total_seats = tpinfos[2].find_all('span')[1].text
        
        if '無參考航班資訊' in source_code:
            start_air = '-'
            goto_start_time = goto_start_airport = goto_arrival_airport = goto_arrival_time = '-'
            back_air = '-'
            back_start_time = back_start_airport = back_arrival_airport = back_arrival_time = '-'
        else:
            # 出發航空
            start_air = tpinfos[3].find('span').text
            # 出發時間, 出發機場, 抵達時間, 抵達機場
            goto_start_time, goto_start_airport, _, goto_arrival_airport, goto_arrival_time = tpinfos[4].find_all('td')[1].text.split()
            # 回國航空
            back_air = tpinfos[5].find('span').text
            # 出發時間, 出發機場, 抵達時間, 抵達機場
            back_start_time, back_start_airport, _, back_arrival_airport, back_arrival_time = tpinfos[6].find_all('td')[1].text.split()

        row = [trip_name, trip_number, price, start_date, back_date,
              empty_seats, total_seats, start_air, goto_start_time,
              goto_start_airport, goto_arrival_time, goto_arrival_airport,
              back_air, back_start_time, back_start_airport, back_arrival_time,
              back_arrival_airport, url]
        row_data=pd.Series(row,columns)
        df = df.append(row_data, ignore_index=True)
    driver.close()
    driver.quit()
    df_filter()
    save_to_local()
    print('done')
    break

1/120
2/120
3/120
4/120
5/120
6/120
7/120
8/120
9/120
10/120
11/120
12/120
13/120
14/120
15/120
16/120
17/120
18/120
19/120
20/120
21/120
22/120
23/120
24/120
25/120
26/120
27/120
28/120
29/120
30/120
31/120
32/120
33/120
34/120
35/120
36/120
37/120
38/120
39/120
40/120
41/120
42/120
43/120
44/120
45/120
46/120
47/120
48/120
49/120
50/120
51/120
52/120
53/120
54/120
55/120
56/120
57/120
58/120
59/120
60/120
61/120
62/120
63/120
64/120
65/120
66/120
67/120
68/120
69/120
70/120
71/120
72/120
73/120
74/120
75/120
76/120
77/120
78/120
79/120
80/120
81/120
82/120
83/120
84/120
85/120
86/120
87/120
88/120
89/120
90/120
91/120
92/120
93/120
94/120
95/120
96/120
97/120
98/120
99/120
100/120
101/120
102/120
103/120
104/120
105/120
106/120
107/120
108/120
109/120
110/120
111/120
112/120
113/120
114/120
115/120
116/120
117/120
118/120
119/120
120/120
